In [42]:
# データの作成
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, PowerTransformer, LabelEncoder
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ステップ1: データの読み込みとDataFrame作成
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["target"] = iris.target

# --- ラベルを文字列に変換 ---
target_map = {0: "setosa", 1: "versicolor", 2: "virginica"}
df["target"] = df["target"].map(target_map)

# --- カラム名変更（扱いやすく） ---
df = df.rename(columns={
    "sepal length (cm)": "sepal_length",
    "sepal width (cm)": "sepal_width",
    "petal length (cm)": "petal_length",
    "petal width (cm)": "petal_width"
})

# --- 欠損の追加（10% ランダムに sepal_wid 欠損） ---
np.random.seed(42)
missing_mask = np.random.rand(len(df)) < 0.1
df.loc[missing_mask, "sepal_width"] = np.nan

# --- 重複データを10%分追加 ---
duplicate_df = df.sample(frac=0.1, random_state=42)
df = pd.concat([df, duplicate_df], ignore_index=True)

# --- 結果をCSVに保存 ---
df.to_csv("iris.csv", index=False)


In [ ]:
# データ分析の基本フローと今回の目標

前回は、データ分析における重要な概念である「過学習」、「次元の呪い」について学びました。  
今回は、実際のデータ分析で必要となる一連の処理の一部を体験します。  
具体的には、予測タスクに至るまでの基本的な流れを確認するために、  
- データの成型（前回の内容の軽い復習）
- データの数値変換（標準化・正規化など）  
を行います。
---

In [ ]:
## データ分析の流れ（全体像）：

1. データの成型（第1回DS研修の宿題の実施範囲）
   - 目的：分析に適した構造に変換するための前処理を行う
   - 手法：merge, concat, rename ,dropna, fillna

2. データの数値変換（<span style="color:red;">今回の宿題のメイン</span>）
   - 目的：モデルが学習しやすい数値空間に変換し、精度と安定性を向上させる
   - 手法：標準化、正規化、対数変換、Box-Cox変換、Yeo-Johnson変換

3. カテゴリ変数の処理
   - 目的：文字列やカテゴリを機械学習で扱える数値に変換する
   - 手法：One-hot encoding, Label encoding, Frequency encoding, Target encoding, Embedding など

4. 特徴量エンジニアリング
   - 目的：予測性能を高めるために、より情報量の多い特徴量を作成する
   - 手法：新規特徴量の作成, 集約統計量の作成, 時系列処理, 非構造データの変換, 外部データの統合

5. 特徴量選択・次元削減
   - 目的：不要または冗長な特徴を減らし、モデルの汎化性能と学習速度を向上させる
   - 手法：多重共線性の確認・削除, モデルベースの選択, 次元削減手法など

6. 特徴量の可視化と評価
   - 目的：特徴量の関係や重要性を可視化して、理解や改善に役立てる
   - 手法：相関マトリクスの可視化, 特徴量の重要度など

7. モデリングと予測
   - 目的：構築した特徴量を使って予測モデルを訓練し、成果を評価する
   - モデル例：回帰、分類、クラスタリング、時系列予測など
   - 評価指標：精度（Accuracy）、RMSE、AUC、F1スコアなどに基づいて性能を検証

### -------------------------------------
### データ前処理演習（宿題）
### iris.csvを使って、以下の指示に従って前処理を行ってください。
### -------------------------------------

In [53]:
## 復習：データの成型

# ライブラリのインポート
import pandas as pd
import numpy as np

# データの読み込み
df = pd.read_csv("iris.csv")

# 【Step 1】データの概要を確認しましょう
print("データの形確認:\n", df.shape)  # データの行数・列数
print("データの先頭5行確認:\n", df.head())  # 先頭5行
print("統計量の確認（describe使用）:\n", df.select_dtypes(exclude=["object"]).describe())  # 数値列の統計量
print("目的変数の分布確認（value_counts使用）:\n", df["target"].value_counts())  # 目的変数の分布
print("重複行の有無:\n", df.duplicated().sum())  # 重複チェック

# 【Step 2】特徴量と目的変数に分割してください
# 特徴量：sepal_length, sepal_width, petal_length, petal_width
# 目的変数：target
# ↓下のコードの空欄を埋めてください
df_x = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]  # 特徴量
df_y = df["target"]  # 目的変数

#以降の処理は、df_xに対してのみ行う

# 【Step 3】各カラム名に「(cm)」という単位を追記してください
df_x = df_x.rename(columns={
    "sepal_length": "sepal_length (cm)",
    "sepal_width": "sepal_width (cm)",
    "petal_length": "petal_length (cm)",
    "petal_width": "petal_width (cm)"
})

# 【Step 4】重複行があるか確認し、あれば削除してください
print("重複している行数：", df_x.duplicated().sum())
df_x = df_x.drop_duplicates()
print("重複している行数：", df_x.duplicated().sum())

# 【Step 5】欠損があるか確認し、sepal_width の欠損を中央値で補完してください
print("欠損の確認:\n", df_x.isnull().sum())
df_x["sepal_width (cm)"] = df_x["sepal_width (cm)"].fillna(df_x["sepal_width (cm)"].median())
print("欠損の確認:\n", df_x.isnull().sum())

# 【Step 6】条件抽出をしてください
# petal_length が 3 以上 かつ petal_width が 1 以上の行のみを抽出
df_x_filtered = df_x.copy()
df_x_filtered = df_x_filtered[(df_x_filtered["petal_length (cm)"] >= 4) & (df_x_filtered["petal_width (cm)"] >= 1)]
print("条件抽出後の行数:", df_x_filtered.shape[0])

# 【Step 7】新しい特徴量 sepal_ratio を作成してください
# → sepal_length ÷ sepal_width
df_x_filtered["sepal_ratio"] = df_x_filtered["sepal_length (cm)"] / df_x_filtered["sepal_width (cm)"]

データの形確認:
 (165, 5)
データの先頭5行確認:
    sepal_length  sepal_width  petal_length  petal_width  target
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa
統計量の確認（describe使用）:
        sepal_length  sepal_width  petal_length  petal_width
count    165.000000   146.000000    165.000000   165.000000
mean       5.851515     3.053425      3.731515     1.187879
std        0.823482     0.425103      1.766794     0.763160
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.500000     0.300000
50%        5.800000     3.000000      4.300000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000
目的変数の分布確認（value_counts使

In [40]:
# データの数値変換

# カラム「"sepal_ratio"」の標準化と正規化を以下の数式を参照して計算する



In [ ]:
### 標準化（Standardization）
- 異なるスケールの特徴量を**平均0・分散1のスケールに変換する処理**

$$
z = \frac{x - \mu}{\sigma}
$$

- $ x $：対象データ
- $ \mu $：対象データの平均
- $ \sigma $：対象データの標準偏差
- $ z $：標準化された値（平均0、標準偏差1）



### 正規化（Min-Max Normalization）
- 元のデータを ** $[0, 1]$ の範囲にスケーリングする手法**

$$
x_{\text{norm}} = \frac{x - x_{\min}}{x_{\max} - x_{\min}}
$$

- $ x $：対象データ
- $ x_{\min} $：対象データの最小値
- $ x_{\max} $：対象データの最大値
- $ x_{\text{norm}} $：正規化された値（[0, 1]の範囲）

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 平均・分散を用いた標準化
# z = (x - μ) / σ
sepal_ratio_mean = df_x["sepal_ratio"].mean()
sepal_ratio_std = df_x["sepal_ratio"].std(ddof=0)
df_x["sepal_ratio_std"] = (df_x["sepal_ratio"] - sepal_ratio_mean) / sepal_ratio_std

# 最小・最大を用いた正規化
# x_norm = (x - x_min) / (x_max - x_min)
sepal_ratio_min = df_x["sepal_ratio"].min()
sepal_ratio_max = df_x["sepal_ratio"].max()
df_x["sepal_ratio_minmax"]  = (df_x["sepal_ratio"] - sepal_ratio_min) / (sepal_ratio_max - sepal_ratio_min)

In [ ]:
## 任意課題：StandardScalerを用いて標準化と正規化を行う

# sklearnを用いた正規化
minmax_scaler = MinMaxScaler()
df_x["sepal_ratio_minmax_sklearn"]= minmax_scaler.fit_transform(df_x[["sepal_ratio"]])

# sklearn を用いた標準化
scaler = StandardScaler()
df_x["sepal_ratio_std_sklearn"] = scaler.fit_transform(df_x[["sepal_ratio"]])

## 任意課題：課題〇と任意課題〇の差分を取れ。また、差が0出ない場合は考察せよ（1e-15の誤差は許容する）


In [ ]:
# 参考（その他の数値変換）
### Box-Cox変換

$$
x' =
\begin{cases}
\frac{x^{\lambda} - 1}{\lambda} & (\lambda \ne 0) \\
\log x & (\lambda = 0)
\end{cases}
$$

- $ x $：元のデータ（必ず $ x > 0 $）
- $ \lambda $：変換パラメータ（データに応じて最適化される）
- $ x' $：変換後のデータ（分布が正規分布に近づく）


### Yeo-Johnson変換

$$
x' =
\begin{cases}
\frac{(x + 1)^\lambda - 1}{\lambda} & (x \geq 0, \lambda \ne 0) \\
\log(x + 1) & (x \geq 0, \lambda = 0) \\
\frac{-(-x + 1)^{2 - \lambda} + 1}{2 - \lambda} & (x < 0, \lambda \ne 2) \\
-\log(-x + 1) & (x < 0, \lambda = 2)
\end{cases}
$$

- $ x $：元のデータ（ゼロや負の値も含んでよい）
- $ \lambda $：変換パラメータ（Box-Coxと同様に推定される）
- $ x' $：変換後のデータ（分布が正規分布に近づく）

In [ ]:
# 【Step 9】（発展）その他の変換（任意）
# 以下のうち1つ以上を適用して、新たな列として保存してください
# - 対数変換（log1p）
# - Box-Cox変換
# - Yeo-Johnson変換